# Homework 13 - Network Compression

Author: Liang-Hsuan Tseng (b07502072@ntu.edu.tw), modified from ML2021-HW13

If you have any questions, feel free to ask: ntu-ml-2022spring-ta@googlegroups.com

[**Link to HW13 Slides**](https://docs.google.com/presentation/d/1nCT9XrInF21B4qQAWuODy5sonKDnpGhjtcAwqa75mVU/edit#slide=id.p)

## Outline

* [Packages](#Packages) - intall some required packages.
* [Dataset](#Dataset) - something you need to know about the dataset.
* [Configs](#Configs) - the configs of the experiments, you can change some hyperparameters here.
* [Architecture_Design](#Architecture_Design) - depthwise and pointwise convolution examples and some useful links.
* [Knowledge_Distillation](#Knowledge_Distillation) - KL divergence loss for knowledge distillation and some useful links.
* [Training](#Training) - training loop implementation modified from HW3.
* [Inference](#Inference) - create submission.csv by using the student_best.ckpt from the previous experiment.



### Packages
First, we need to import some useful packages. If the torchsummary package are not intalled, please install it via `pip install torchsummary`

In [1]:
# Import some useful packages for this homework
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset # "ConcatDataset" and "Subset" are possibly useful
from torchvision.datasets import DatasetFolder, VisionDataset
from torchsummary import summary
from tqdm import tqdm
import random

# !nvidia-smi # list your current GPU

### Configs
In this part, you can specify some variables and hyperparameters as your configs.

In [2]:
cfg = {
    'dataset_root': './food11-hw13',
    'save_dir': './outputs',
    'exp_name': "resnet",
    'batch_size': 32,
    'lr': 5e-4,
    'seed': 5201314,
    'loss_fn_type': 'KD', # simple baseline: CE, medium baseline: KD. See the Knowledge_Distillation part for more information.
    'weight_decay': 1e-4,
    'grad_norm_max': 10,
    'n_epochs': 300, # train more steps to pass the medium baseline.
    'patience': 300,
    'MIXUP': True,
    'TTA': True,
}

In [3]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [4]:
same_seeds(cfg['seed'])                                     # set a random seed for reproducibility

save_path = os.path.join(cfg['save_dir'], cfg['exp_name'])  # create saving directory
os.makedirs(save_path, exist_ok=True)

# define simple logging functionality
log_fw = open(f"{save_path}/log.txt", 'w') # open log file to save log outputs
def log(text):     # define a logging function to trace the training process
    print(text)
    log_fw.write(str(text)+'\n')
    log_fw.flush()

log(cfg)  # log your configs to the log file

# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"
log(f"device: {device}")

{'dataset_root': './food11-hw13', 'save_dir': './outputs', 'exp_name': 'resnet', 'batch_size': 32, 'lr': 0.0005, 'seed': 5201314, 'loss_fn_type': 'KD', 'weight_decay': 0.0001, 'grad_norm_max': 10, 'n_epochs': 300, 'patience': 300, 'MIXUP': True, 'TTA': True}
device: cuda


### Dataset
We use Food11 dataset for this homework, which is similar to homework3. But remember, Please DO NOT utilize the dataset of HW3. We've modified the dataset, so you should only access the dataset by loading it in this kaggle notebook or through the links provided in the HW13 colab notebooks.

In [5]:
# # fetch and download the dataset from github (about 1.12G)
# !wget https://github.com/virginiakm1988/ML2022-Spring/raw/main/HW13/food11-hw13.tar.gz 
# ## backup links:

# # !wget https://github.com/andybi7676/ml2022spring-hw13/raw/main/food11-hw13.tar.gz
# # !gdown '1ijKoNmpike_yjUw8SWRVVWVoMOXXqycj' --output food11-hw13.tar.gz

In [6]:
# # extract the data
# !tar -xzf ./food11-hw13.tar.gz # Could take some time
# # !tar -xzvf ./food11-hw13.tar.gz # use this command if you want to checkout the whole process.

In [7]:
for dirname, _, filenames in os.walk('./food11-hw13'):
    if len(filenames) > 0:
        print(f"{dirname}: {len(filenames)} files.") # Show the file amounts in each split.

./food11-hw13: 1 files.
./food11-hw13/validation: 3430 files.
./food11-hw13/training: 9866 files.
./food11-hw13/evaluation: 3347 files.


Next, specify train/test transform for image data augmentation.
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms. You can also apply the knowledge or experience you learned in HW3.

In [8]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# define training/testing transforms
test_tfm = transforms.Compose([
    # It is not encouraged to modify this part if you are using the provided teacher model. This transform is stardard and good enough for testing.
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    normalize,
])

train_tfm = transforms.Compose([
    # add some useful transform or augmentation here, according to your experience in HW3.
    transforms.Resize((256, 256)),  # You can change this
    transforms.CenterCrop((224, 224)), # You can change this, but be aware of that the given teacher model's input size is 224.
    # The training input size of the provided teacher model is (3, 224, 224).
    # Thus, Input size other then 224 might hurt the performance. please be careful.

    # Use transform
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=(-30, 30)),
    transforms.ToTensor(),
    normalize,
])

In [9]:
class FoodDataset(Dataset):
    def __init__(self, path, tfm=test_tfm, files = None):
        super().__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im, label

In [10]:
# Form train/valid dataloaders
train_set = FoodDataset(os.path.join(cfg['dataset_root'],"training"), tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=cfg['batch_size'], shuffle=True, num_workers=4, pin_memory=True)

valid_set = FoodDataset(os.path.join(cfg['dataset_root'], "validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=4, pin_memory=True)

One ./food11-hw13/training sample ./food11-hw13/training/0_0.jpg
One ./food11-hw13/validation sample ./food11-hw13/validation/0_0.jpg


### Architecture_Design

In this homework, you have to design a smaller network and make it perform well. Apparently, a well-designed architecture is crucial for such task. Here, we introduce the depthwise and pointwise convolution. These variants of convolution are some common techniques for architecture design when it comes to network compression.

<img src="https://i.imgur.com/LFDKHOp.png" width=400px>

* explanation of depthwise and pointwise convolutions:
    * [prof. Hung-yi Lee's slides(p.24~p.30, especially p.28)](https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/tiny_v7.pdf)

In [12]:
# Example implementation of Depthwise and Pointwise Convolution 
def deepwise_pointwise_conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    return nn.Sequential(
        nn.Conv2d(in_channels, in_channels, kernel_size, stride=stride, padding=padding, groups=in_channels),   # depthwise convolution (in_channels = out_channels)
        nn.BatchNorm2d(in_channels),
        nn.ReLU(),
        nn.Conv2d(in_channels, out_channels, kernel_size=1),                                                    # pointwise convolution (kernel size = (1, 1))
    )

* other useful techniques
    * [group convolution](https://www.researchgate.net/figure/The-transformations-within-a-layer-in-DenseNets-left-and-CondenseNets-at-training-time_fig2_321325862) (Actually, depthwise convolution is a specific type of group convolution)
    * [SqueezeNet](!https://arxiv.org/abs/1602.07360)
    * [MobileNet](!https://arxiv.org/abs/1704.04861)
    * [ShuffleNet](!https://arxiv.org/abs/1707.01083)
    * [Xception](!https://arxiv.org/abs/1610.02357)
    * [GhostNet](!https://arxiv.org/abs/1911.11907)


After introducing depthwise and pointwise convolutions, let's define the **student network architecture**. Here, we have a very simple network formed by some regular convolution layers and pooling layers. You can replace the regular convolution layers with the depthwise and pointwise convolutions. In this way, you can further increase the depth or the width of your network architecture.

**Model Architecture Reference**  
https://github.com/peterliht/knowledge-distillation-pytorch/blob/ef06124d67a98abcb3a5bc9c81f7d0f1f016a7ef/model/resnet.py

In [13]:
# Define your student network here. You have to copy-paste this code block to HW13 GradeScope before deadline.
# We will use your student network definition to evaluate your results(including the total parameter amount).

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = deepwise_pointwise_conv(in_planes, planes, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = deepwise_pointwise_conv(planes, planes, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                deepwise_pointwise_conv(in_planes, self.expansion*planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = deepwise_pointwise_conv(in_planes, planes, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = deepwise_pointwise_conv(planes, planes, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = deepwise_pointwise_conv(planes, self.expansion*planes, kernel_size=1)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                deepwise_pointwise_conv(in_planes, self.expansion*planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class StudentNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=11):
        super().__init__()

        # ---------- TODO ----------
        # Modify your model architecture

        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.linear = nn.Linear(128*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = F.adaptive_avg_pool2d(F.avg_pool2d(out, 4), 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def get_student_model(): # This function should have no arguments so that we can get your student network by directly calling it.
    # you can modify or do anything here, just remember to return an nn.Module as your student network.  
    return StudentNet(BasicBlock, [2, 2])

# End of definition of your student model and the get_student_model API
# Please copy-paste the whole code block, including the get_student_model function.

After specifying the student network architecture, please use `torchsummary` package to get information about the network and verify the total number of parameters. Note that the total params of your student network should not exceed the limit (`Total params` in `torchsummary` ≤ 100,000). 

In [14]:
# DO NOT modify this block and please make sure that this block can run sucessfully. 
student_model = get_student_model()
summary(student_model, (3, 224, 224), device='cpu')
# You have to copy&paste the results of this block to HW13 GradeScope. 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,728
       BatchNorm2d-2         [-1, 64, 224, 224]             128
            Conv2d-3         [-1, 64, 224, 224]             640
       BatchNorm2d-4         [-1, 64, 224, 224]             128
              ReLU-5         [-1, 64, 224, 224]               0
            Conv2d-6         [-1, 64, 224, 224]           4,160
       BatchNorm2d-7         [-1, 64, 224, 224]             128
            Conv2d-8         [-1, 64, 224, 224]             640
       BatchNorm2d-9         [-1, 64, 224, 224]             128
             ReLU-10         [-1, 64, 224, 224]               0
           Conv2d-11         [-1, 64, 224, 224]           4,160
      BatchNorm2d-12         [-1, 64, 224, 224]             128
       BasicBlock-13         [-1, 64, 224, 224]               0
           Conv2d-14         [-1, 64, 2

In [15]:
# Load provided teacher model (model architecture: resnet18, num_classes=11, test-acc ~= 89.9%)
teacher_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False, num_classes=11).to(device)
# load state dict
teacher_ckpt_path = os.path.join(cfg['dataset_root'], "resnet18_teacher.ckpt")
teacher_model.load_state_dict(torch.load(teacher_ckpt_path, map_location='cpu'))
# Now you already know the teacher model's architecture. You can take advantage of it if you want to pass the strong or boss baseline. 
# Source code of resnet in pytorch: (https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py)
# You can also see the summary of teacher model. There are 11,182,155 parameters totally in the teacher model
# summary(teacher_model, (3, 224, 224), device='cpu')

Using cache found in /home/tienyi/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

### Knowledge_Distillation

<img src="https://i.imgur.com/H2aF7Rv.png=100x" width="400px">

Since we have a learned big model, let it teach the other small model. In implementation, let the training target be the prediction of big model instead of the ground truth.

**Why it works?**
* If the data is not clean, then the prediction of big model could ignore the noise of the data with wrong labeled.
* There might have some relations between classes, so soft labels from teacher model might be useful. For example, Number 8 is more similar to 6, 9, 0 than 1, 7.


**How to implement?**
* $Loss = \alpha T^2 \times KL(p || q) + (1-\alpha)(\text{Original Cross Entropy Loss}), \text{where } p=softmax(\frac{\text{student's logits}}{T}), \text{and } q=softmax(\frac{\text{teacher's logits}}{T})$
* very useful link: [pytorch docs of KLDivLoss with examples](!https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html)
* original paper: [Distilling the Knowledge in a Neural Network](!https://arxiv.org/abs/1503.02531)

In [16]:
# Ref: https://github.com/facebookresearch/mixup-cifar10/blob/main/train.py
def mixup_data(x, y, alpha=1.0, device='cuda'):
    """
    Returns mixed inputs, pairs of targets, and lambda.
    Arguments:
        x      : A batch of data.
        y      : A batch of labels.
        alpha  : Beta distribution.
        device : Your device('cuda' / 'cpu').
    Returns:
        mixed_x: Mix up data(Tensor).
        y_a    : One of label.
        y_b    : The other label.
        lam    : Mixup ratio.
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]        # Get batch size.
    if device == 'cuda':            # Support cuda
        index = torch.randperm(batch_size).to(device)
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """
    Criterion for mixup data.
    Arguments:
        criterion: Loss function.
        pred     : Prediction of model.
        y_a      : One of label.
        y_b      : The other label.
        lam      : Mixup ratio.
    Returns:
        Tensor   : Loss of mixup data.
    """
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [17]:
# Implement the loss function with KL divergence loss for knowledge distillation.
# You also have to copy-paste this whole block to HW13 GradeScope. 
def loss_fn_kd(student_logits, teacher_logits, y_a, y_b=None, lam=0, alpha=0.3, temperature=2.0):
    # ------------TODO-------------
    # Refer to the above formula and finish the loss function for knowkedge distillation using KL divergence loss and CE loss.
    # If you have no idea, please take a look at the provided useful link above.

    p = F.softmax(student_logits / temperature, dim=-1)
    q = F.softmax(teacher_logits / temperature, dim=-1)
    kl_loss = nn.KLDivLoss(reduction='batchmean')(p, q)
    loss_fn = nn.CrossEntropyLoss()
    if cfg['MIXUP'] and y_b is not None:
        ce_loss = mixup_criterion(loss_fn, student_logits, y_a, y_b, lam)
    else:
        ce_loss = loss_fn(student_logits, y_a)
    loss = (alpha*temperature**2) * kl_loss + (1 - alpha) * ce_loss
    return loss

In [18]:
# choose the loss function by the config
if cfg['loss_fn_type'] == 'CE':
    # For the classification task, we use cross-entropy as the default loss function.
    loss_fn = nn.CrossEntropyLoss() # loss function for simple baseline.

if cfg['loss_fn_type'] == 'KD': # KD stands for knowledge distillation
    loss_fn = loss_fn_kd # implement loss_fn_kd for the report question and the medium baseline.

# You can also adopt other types of knowledge distillation techniques for strong and boss baseline, but use function name other than `loss_fn_kd`
# For example:
# def loss_fn_custom_kd():
#     pass
# if cfg['loss_fn_type'] == 'custom_kd':
#     loss_fn = loss_fn_custom_kd

# The number of training epochs and patience.
n_epochs = cfg['n_epochs']
patience = cfg['patience'] # If no improvement in 'patience' epochs, early stop

### Training
implement training loop for simple baseline, feel free to modify it.

In [19]:
# Initialize a model, and put it on the device specified.
student_model.to(device)
teacher_model.to(device) # MEDIUM BASELINE

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(student_model.parameters(), lr=cfg['lr'], weight_decay=cfg['weight_decay']) 

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0.0

teacher_model.eval()  # MEDIUM BASELINE
for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    student_model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []
    train_lens = []

    for X, y in train_loader:

        # A batch consists of image data and corresponding labels.
        X = X.to(device)
        y = y.to(device)

        if cfg['MIXUP']:
            X, y_a, y_b, lam = mixup_data(X, y, alpha=1, device=device)
        else:
            y_a, y_b, lam = y, None, 0

        # Forward the data. (Make sure data and model are on the same device.)
        with torch.no_grad():                       # MEDIUM BASELINE
            teacher_logits = teacher_model(X)    # MEDIUM BASELINE

        logits = student_model(X)

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        if cfg['loss_fn_type'] == 'KD':
            loss = loss_fn(logits, teacher_logits, y_a, y_b, lam)   # MEDIUM BASELINE
        elif cfg['loss_fn_type'] == 'CE':
            loss = loss_fn(logits, y)                               # SIMPLE BASELINE

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(student_model.parameters(), max_norm=cfg['grad_norm_max'])

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == y).float().sum()

        # Record the loss and accuracy.
        train_batch_len = len(X)
        train_loss.append(loss.item() * train_batch_len)
        train_accs.append(acc)
        train_lens.append(train_batch_len)
        
    train_loss = sum(train_loss) / sum(train_lens)
    train_acc = sum(train_accs) / sum(train_lens)

    # Print the information.
    log(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    student_model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []
    valid_lens = []

    # Iterate the validation set by batches.
    for X, y in valid_loader:

        # A batch consists of image data and corresponding labels.
        X = X.to(device)
        y = y.to(device)

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = student_model(X).squeeze()
            teacher_logits = teacher_model(X)        # MEDIUM BASELINE

        # We can still compute the loss (but not the gradient).
        if cfg['loss_fn_type'] == 'KD':
            loss = loss_fn(logits, teacher_logits, y)   # MEDIUM BASELINE
        elif cfg['loss_fn_type'] == 'CE':
            loss = loss_fn(logits, y)                   # SIMPLE BASELINE

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == y).float().sum()

        # Record the loss and accuracy.
        batch_len = len(X)
        valid_loss.append(loss.item() * batch_len)
        valid_accs.append(acc)
        valid_lens.append(batch_len)
        # break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / sum(valid_lens)
    valid_acc = sum(valid_accs) / sum(valid_lens)

    # update logs
    
    if valid_acc > best_acc:
        log(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        log(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        log(f"Best model found at epoch {epoch + 1}, saving model")
        torch.save(student_model.state_dict(), f"{save_path}/student_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            log(f"No improvment {patience} consecutive epochs, early stopping")
            break
log("Finish training")
log_fw.close()

[ Train | 001/300 ] loss = 0.53829, acc = 0.23728
[ Valid | 001/300 ] loss = 0.57285, acc = 0.36414 -> best
Best model found at epoch 1, saving model
[ Train | 002/300 ] loss = 0.46757, acc = 0.25532
[ Valid | 002/300 ] loss = 0.54081, acc = 0.37318 -> best
Best model found at epoch 2, saving model
[ Train | 003/300 ] loss = 0.43914, acc = 0.27083
[ Valid | 003/300 ] loss = 0.46822, acc = 0.41778 -> best
Best model found at epoch 3, saving model
[ Train | 004/300 ] loss = 0.41182, acc = 0.27965
[ Valid | 004/300 ] loss = 0.44254, acc = 0.42041 -> best
Best model found at epoch 4, saving model
[ Train | 005/300 ] loss = 0.35025, acc = 0.28644
[ Valid | 005/300 ] loss = 0.41307, acc = 0.41516
[ Train | 006/300 ] loss = 0.34492, acc = 0.30316
[ Valid | 006/300 ] loss = 0.33348, acc = 0.45918 -> best
Best model found at epoch 6, saving model
[ Train | 007/300 ] loss = 0.29077, acc = 0.31735
[ Valid | 007/300 ] loss = 0.28644, acc = 0.47872 -> best
Best model found at epoch 7, saving model


### Inference
load the best model of the experiment and generate submission.csv

In [20]:
import gc
def test_time_augmentation(config, model, classes=11, alpha=0.5, numAugment=5, test_tfm=None, test_aug_tfm=None):
    """
    Test Time Augmentation.
    Arguments:
        config        : Configuration dictionary.
        model         : Your model.
        classes       : Output classes.
        alpha         : Weight of prediction without augmentation, range (0, 1].
        numAugment    : Number of augmentation times.
        test_tfm      : Testing transform.
        test_aug_tfm  : Testing transform with augmentation.
    Returns:
        Numpy array   : If use K-Fold, return the score after the weighted-sum;
                        otherwise, return the prediction.
    """

    """ Evaluation """
    model.eval()

    test_pred = np.empty((0, classes))                                  # Prediction of the test dataset without augmentation.

    # First to gain the test_loader without augmentation
    test_set = FoodDataset(os.path.join(config['dataset_root'], "evaluation"), tfm=test_tfm)
    test_loader = DataLoader(test_set, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)

    with torch.no_grad():
        for data, _ in tqdm(test_loader):
            data = data.to(device)
            pred = model(data)
            test_pred = np.vstack((test_pred, pred.detach().cpu().numpy()))

    print(test_pred.argmax(axis=-1))
    test_pred = test_pred * alpha                                       # Weight of prediction without augmentation.

    del test_set, test_loader
    gc.collect()

    # Second to gain the test_loader with augmentation
    test_set = FoodDataset(os.path.join(cfg['dataset_root'], "evaluation"), tfm=test_aug_tfm)
    test_loader = DataLoader(test_set, batch_size=config['batch_size'], shuffle=False, num_workers=4, pin_memory=True)

    numTestDataset = len(test_loader.sampler)                           # Number of test dtatset.
    augment_preds = np.empty((numTestDataset, classes, 0))              # Prediction of the test dataset with augmentation.

    with torch.no_grad():
        for idx in range(numAugment):
            predictions = np.empty((0, classes))
            for data, _ in tqdm(test_loader):
                data = data.to(device)
                pred = model(data)
                predictions = np.vstack((predictions, pred.detach().cpu().numpy()))
            augment_preds = np.dstack((augment_preds, predictions))

    mean_augment_preds = np.mean(augment_preds, axis=2) * (1 - alpha)   # Weight of prediction with augmentation.
    score = test_pred + mean_augment_preds
    prediction = score.argmax(axis=-1)
    return prediction

In [21]:
kwargs = {
    'alpha': 0.5, 
    'numAugment': 5,
    'test_tfm': test_tfm,
    'test_aug_tfm': train_tfm
}

# Load model from {exp_name}/student_best.ckpt
student_model_best = get_student_model() # get a new student model to avoid reference before assignment.
ckpt_path = f"{save_path}/student_best.ckpt" # the ckpt path of the best student model.
student_model_best.load_state_dict(torch.load(ckpt_path, map_location='cpu')) # load the state dict and set it to the student model
student_model_best.to(device) # set the student model to device

if cfg['TTA']:
    prediction = test_time_augmentation(cfg, student_model_best, classes=11, **kwargs)
else:
    test_set = FoodDataset(os.path.join(cfg['dataset_root'], "evaluation"), tfm=test_tfm)
    test_loader = DataLoader(test_set, batch_size=cfg['batch_size'], shuffle=False, num_workers=0, pin_memory=True)
    # Start evaluate
    student_model_best.eval()
    prediction = []
    with torch.no_grad():
        for data, _ in tqdm(test_loader):
            test_pred = student_model_best(data.to(device))
            test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
            prediction += test_label.squeeze().tolist()

One ./food11-hw13/evaluation sample ./food11-hw13/evaluation/0000.jpg


100%|██████████| 105/105 [00:06<00:00, 15.42it/s]


[3 4 4 ... 9 3 4]
One ./food11-hw13/evaluation sample ./food11-hw13/evaluation/0000.jpg


100%|██████████| 105/105 [00:06<00:00, 15.35it/s]


In [22]:
def pad4(i):
    return "0"*(4-len(str(i))) + str(i)

# Save prediction results
ids = [pad4(i) for i in range(0, len(prediction))]
categories = prediction

df = pd.DataFrame()
df['Id'] = ids
df['Category'] = categories
df.to_csv(f"{save_path}/submission_TTA.csv", index=False) # now you can download the submission.csv and upload it to the kaggle competition.

> Don't forget to answer the report questions on GradeScope! 

0.85956